# Interactive Tax Policy Analysis

このノートブックでは、インタラクティブなウィジェットを使用して、税制変更の影響をリアルタイムで分析できます。

In [1]:
# 必要なライブラリのインポート
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML

# DSGEモデルのインポート
from src.dsge_model import DSGEModel, ModelParameters, load_model
from src.tax_simulator import EnhancedTaxSimulator, TaxReform

# プロット設定
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. モデルの初期化

In [2]:
# モデルの読み込み
model = load_model('../config/parameters.json')
simulator = EnhancedTaxSimulator(model)

# ベースラインの定常状態を取得
baseline_ss = model.steady_state.to_dict()

print("モデルの初期化が完了しました。")
print(f"ベースライン税率:")
print(f"  消費税: {model.params.tau_c:.1%}")
print(f"  所得税: {model.params.tau_l:.1%}")
print(f"  資本所得税: {model.params.tau_k:.1%}")
print(f"  法人税: {model.params.tau_f:.1%}")

AttributeError: 'NoneType' object has no attribute 'to_dict'

## 2. インタラクティブ税制シミュレーター

In [ ]:
# ウィジェットの作成
tau_c_slider = widgets.FloatSlider(
    value=0.10,
    min=0.0,
    max=0.30,
    step=0.01,
    description='消費税率:',
    continuous_update=False
)

tau_l_slider = widgets.FloatSlider(
    value=0.20,
    min=0.0,
    max=0.50,
    step=0.01,
    description='所得税率:',
    continuous_update=False
)

tau_k_slider = widgets.FloatSlider(
    value=0.25,
    min=0.0,
    max=0.50,
    step=0.01,
    description='資本所得税率:',
    continuous_update=False
)

tau_f_slider = widgets.FloatSlider(
    value=0.30,
    min=0.0,
    max=0.50,
    step=0.01,
    description='法人税率:',
    continuous_update=False
)

implementation_dropdown = widgets.Dropdown(
    options=['permanent', 'temporary', 'phased'],
    value='permanent',
    description='実施方法:'
)

periods_slider = widgets.IntSlider(
    value=40,
    min=20,
    max=100,
    step=10,
    description='期間:',
    continuous_update=False
)

# 結果表示用の出力エリア
output = widgets.Output()

# シミュレーション実行関数
def run_simulation(tau_c, tau_l, tau_k, tau_f, implementation, periods):
    with output:
        output.clear_output(wait=True)
        
        # 税制改革の設定
        reform = TaxReform(
            name="カスタム税制改革",
            tau_c=tau_c,
            tau_l=tau_l,
            tau_k=tau_k,
            tau_f=tau_f,
            implementation=implementation
        )
        
        try:
            # シミュレーション実行
            results = simulator.simulate_reform(reform, periods=periods)
            
            # インタラクティブプロットの作成
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('GDP', '消費', '投資', '労働時間')
            )
            
            # 各変数のプロット
            variables = ['Y', 'C', 'I', 'L']
            positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
            
            for var, (row, col) in zip(variables, positions):
                if var in results.paths.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=results.paths.index,
                            y=results.paths[var],
                            name=var,
                            mode='lines'
                        ),
                        row=row, col=col
                    )
            
            fig.update_layout(
                height=600,
                title_text="税制改革シミュレーション結果",
                showlegend=False
            )
            
            fig.show()
            
            # 定常状態への影響を表示
            print("\n=== 定常状態への影響 ===")
            new_ss = results.new_steady_state
            
            for var in ['Y', 'C', 'I', 'L']:
                if hasattr(new_ss, var):
                    old_val = baseline_ss[var]
                    new_val = getattr(new_ss, var)
                    change = (new_val - old_val) / old_val * 100
                    print(f"{var}: {change:+.2f}%")
            
            # 厚生分析
            if hasattr(results, 'welfare_change'):
                print(f"\n厚生変化（消費等価変分）: {results.welfare_change:.2%}")
                
        except Exception as e:
            print(f"エラーが発生しました: {e}")

# インタラクティブウィジェットの設定
interactive_plot = widgets.interactive(
    run_simulation,
    tau_c=tau_c_slider,
    tau_l=tau_l_slider,
    tau_k=tau_k_slider,
    tau_f=tau_f_slider,
    implementation=implementation_dropdown,
    periods=periods_slider
)

# ウィジェットの表示
display(interactive_plot, output)

## 3. 税収最大化分析

In [ ]:
# ラッファー曲線の描画
def compute_laffer_curve(tax_type='consumption', tax_rates=None):
    if tax_rates is None:
        tax_rates = np.linspace(0, 0.5, 21)
    
    revenues = []
    gdp_levels = []
    
    for rate in tax_rates:
        # パラメータ設定
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        # 税率を変更
        if tax_type == 'consumption':
            test_params.tau_c = rate
        elif tax_type == 'income':
            test_params.tau_l = rate
        elif tax_type == 'capital':
            test_params.tau_k = rate
        elif tax_type == 'corporate':
            test_params.tau_f = rate
        
        # モデル計算
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            revenues.append(ss.T_total_revenue)
            gdp_levels.append(ss.Y)
        except:
            revenues.append(np.nan)
            gdp_levels.append(np.nan)
    
    return tax_rates, revenues, gdp_levels

# インタラクティブラッファー曲線
tax_type_dropdown = widgets.Dropdown(
    options=['consumption', 'income', 'capital', 'corporate'],
    value='consumption',
    description='税種別:'
)

laffer_output = widgets.Output()

def plot_laffer_curve(tax_type):
    with laffer_output:
        laffer_output.clear_output(wait=True)
        
        print(f"{tax_type}税のラッファー曲線を計算中...")
        rates, revenues, gdp_levels = compute_laffer_curve(tax_type)
        
        # プロット作成
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('税収', 'GDP')
        )
        
        # 税収のプロット
        fig.add_trace(
            go.Scatter(
                x=rates * 100,
                y=revenues,
                mode='lines+markers',
                name='税収'
            ),
            row=1, col=1
        )
        
        # GDPのプロット
        fig.add_trace(
            go.Scatter(
                x=rates * 100,
                y=gdp_levels,
                mode='lines+markers',
                name='GDP'
            ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_text="税率 (%)", row=1, col=1)
        fig.update_xaxes(title_text="税率 (%)", row=1, col=2)
        fig.update_yaxes(title_text="税収", row=1, col=1)
        fig.update_yaxes(title_text="GDP", row=1, col=2)
        
        fig.update_layout(
            height=400,
            title_text=f"{tax_type}税のラッファー曲線",
            showlegend=False
        )
        
        fig.show()
        
        # 最大税収をもたらす税率を表示
        max_revenue_idx = np.nanargmax(revenues)
        optimal_rate = rates[max_revenue_idx]
        max_revenue = revenues[max_revenue_idx]
        
        print(f"\n最大税収をもたらす税率: {optimal_rate:.1%}")
        print(f"最大税収: {max_revenue:.3f}")

interactive_laffer = widgets.interactive(
    plot_laffer_curve,
    tax_type=tax_type_dropdown
)

display(interactive_laffer, laffer_output)

## 4. 動的経路の3D可視化

In [ ]:
# 3D動的経路の可視化
def plot_3d_dynamics():
    # 複数の税制シナリオを比較
    scenarios = [
        {'name': 'ベースライン', 'tau_c': 0.10, 'tau_l': 0.20},
        {'name': '消費税増税', 'tau_c': 0.15, 'tau_l': 0.20},
        {'name': '所得税減税', 'tau_c': 0.10, 'tau_l': 0.15},
        {'name': '複合改革', 'tau_c': 0.15, 'tau_l': 0.15}
    ]
    
    fig = go.Figure()
    
    for scenario in scenarios:
        reform = TaxReform(
            name=scenario['name'],
            tau_c=scenario['tau_c'],
            tau_l=scenario['tau_l'],
            implementation='permanent'
        )
        
        results = simulator.simulate_reform(reform, periods=40)
        
        # 3次元プロット（GDP, 消費, 投資）
        fig.add_trace(go.Scatter3d(
            x=results.paths['Y'],
            y=results.paths['C'],
            z=results.paths['I'],
            mode='lines+markers',
            name=scenario['name'],
            marker=dict(size=3),
            line=dict(width=3)
        ))
    
    fig.update_layout(
        title='税制改革の動的経路（3D）',
        scene=dict(
            xaxis_title='GDP',
            yaxis_title='消費',
            zaxis_title='投資'
        ),
        height=700
    )
    
    fig.show()

# 3Dプロットの実行
plot_3d_dynamics()

## 5. 税制改革の最適化

In [ ]:
# 税制最適化問題
def optimize_tax_mix(objective='welfare', constraint='revenue_neutral'):
    """
    目的関数を最大化する税制ミックスを探索
    """
    from scipy.optimize import minimize
    
    # ベースラインの税収を取得
    baseline_revenue = model.steady_state.T_total_revenue
    
    def objective_function(x):
        tau_c, tau_l = x
        
        # パラメータ設定
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        test_params.tau_c = tau_c
        test_params.tau_l = tau_l
        
        # モデル計算
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            
            if objective == 'welfare':
                # 厚生（消費）を最大化
                return -ss.C  # 最小化問題なので符号を反転
            elif objective == 'gdp':
                return -ss.Y
            elif objective == 'employment':
                return -ss.L
        except:
            return 1e10  # 計算失敗時のペナルティ
    
    def constraint_function(x):
        tau_c, tau_l = x
        
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        test_params.tau_c = tau_c
        test_params.tau_l = tau_l
        
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            if constraint == 'revenue_neutral':
                return ss.T_total_revenue - baseline_revenue
        except:
            return -1e10
    
    # 最適化実行
    x0 = [model.params.tau_c, model.params.tau_l]
    bounds = [(0, 0.3), (0, 0.5)]
    
    if constraint:
        constraints = {'type': 'eq', 'fun': constraint_function}
        result = minimize(objective_function, x0, bounds=bounds, constraints=constraints, method='SLSQP')
    else:
        result = minimize(objective_function, x0, bounds=bounds, method='L-BFGS-B')
    
    return result

# 最適化結果の表示
print("税制最適化の実行...")
result = optimize_tax_mix(objective='welfare', constraint='revenue_neutral')

if result.success:
    optimal_tau_c, optimal_tau_l = result.x
    print(f"\n最適な税制ミックス（厚生最大化、税収中立）:")
    print(f"  消費税率: {optimal_tau_c:.1%}")
    print(f"  所得税率: {optimal_tau_l:.1%}")
    print(f"\n現行税制との比較:")
    print(f"  消費税: {model.params.tau_c:.1%} → {optimal_tau_c:.1%}")
    print(f"  所得税: {model.params.tau_l:.1%} → {optimal_tau_l:.1%}")
else:
    print("最適化に失敗しました。")

## 6. 感応度分析ヒートマップ

In [ ]:
# 2つの税率の組み合わせによる影響をヒートマップで表示
def create_sensitivity_heatmap(variable='Y'):
    tau_c_range = np.linspace(0.05, 0.20, 10)
    tau_l_range = np.linspace(0.10, 0.30, 10)
    
    results = np.zeros((len(tau_l_range), len(tau_c_range)))
    
    for i, tau_l in enumerate(tau_l_range):
        for j, tau_c in enumerate(tau_c_range):
            test_params = ModelParameters()
            for attr in dir(model.params):
                if not attr.startswith('_'):
                    setattr(test_params, attr, getattr(model.params, attr))
            
            test_params.tau_c = tau_c
            test_params.tau_l = tau_l
            
            test_model = DSGEModel(test_params)
            try:
                ss = test_model.compute_steady_state()
                baseline_val = getattr(model.steady_state, variable)
                new_val = getattr(ss, variable)
                results[i, j] = (new_val - baseline_val) / baseline_val * 100
            except:
                results[i, j] = np.nan
    
    # ヒートマップの作成
    fig = go.Figure(data=go.Heatmap(
        z=results,
        x=tau_c_range * 100,
        y=tau_l_range * 100,
        colorscale='RdBu',
        zmid=0,
        text=np.round(results, 1),
        texttemplate='%{text}%',
        textfont={"size": 10},
        colorbar=dict(title="変化率 (%)")
    ))
    
    fig.update_layout(
        title=f'{variable}の感応度分析',
        xaxis_title='消費税率 (%)',
        yaxis_title='所得税率 (%)',
        width=700,
        height=600
    )
    
    # 現行税制の位置をマーク
    fig.add_scatter(
        x=[model.params.tau_c * 100],
        y=[model.params.tau_l * 100],
        mode='markers',
        marker=dict(size=15, color='red', symbol='x'),
        name='現行税制'
    )
    
    fig.show()

# ヒートマップの表示
variable_dropdown = widgets.Dropdown(
    options=['Y', 'C', 'I', 'L', 'T_total_revenue'],
    value='Y',
    description='変数:'
)

interactive_heatmap = widgets.interactive(
    create_sensitivity_heatmap,
    variable=variable_dropdown
)

display(interactive_heatmap)

## まとめ

このインタラクティブノートブックでは、以下の分析ツールを提供しました：

1. **リアルタイム税制シミュレーター**: スライダーで税率を調整し、即座に経済への影響を確認
2. **ラッファー曲線分析**: 各税種別の税収最大化ポイントを特定
3. **3D動的経路分析**: 複数シナリオの経済変数の動的経路を3次元で可視化
4. **税制最適化**: 厚生や成長を最大化する最適な税制ミックスを探索
5. **感応度分析**: 2つの税率の組み合わせによる影響をヒートマップで表示

これらのツールを活用することで、政策立案者は税制改革の影響を多角的に分析し、より良い政策決定を行うことができます。